<a href="https://colab.research.google.com/github/ykato27/Scraping/blob/main/Scraping_%E5%AE%B6%E8%B3%83%E8%AA%BF%E6%9F%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# モジュールのimport
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm as tqdm #for文使うときは必ずつける。
import pandas as pd
import time

In [ ]:
# スクレイピングした要素を入れる空のリストを作成。
house_name=[]
price=[]
kanrihi=[]
sikikin=[]
reikin=[]
hosyokin=[]
sikibiki=[]
address=[]
nearest_station=[]
layout=[]
area=[]
age=[]
floor=[]
direction=[]
house_type=[]
construction=[]
floor_max=[]
insurance=[]
parking=[]
separate_bath=[]
elevator=[]
pet=[]
washing_machin=[]
auto_lock=[]
washroom=[]
url_syosai_lists=[]

# urlの最後に&page=1をつけると１ページ目、&page=2だと２ページ目が表示されるので、まずページ数をfor文で回す。
# rangeの80はだいたい75ページだったから。スクレイピング中にページ数が変わるかもしれないので大体で良い。

for h in tqdm(range(1,80)):
    try:
        h=str(h)
        # 桶川市
        url="https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ra=011&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&ek=040506730&rn=0405" + "&page=" + h
        urls=requests.get(url)
        urls.encoding=urls.apparent_encoding #これ地味に時間かかる
        soup=BeautifulSoup(urls.content,"html.parser")
        syosai=soup.find_all("a",class_="js-cassette_link_href cassetteitem_other-linktext") #詳細を見るのリンクを全て取得。

        for i in range(len(syosai)): #上記で取得したsyosaiを１つずつ取り出し、そｎi番目のhrefタグを取り出し、url_syosaiにする。
            id_href=syosai[i].get("href")# URLは"href"タグにある。
            url_syosai="https://suumo.jp" + id_href
            url_syosai_lists.append(url_syosai)
        time.sleep(1) #サーバーに負荷がかかるので１秒待つ。

    except IndexError: #ページ数はだいたいなのでエラーが出ても続けるようにする。
        continue

 28%|██▊       | 22/79 [01:48<04:40,  4.92s/it]

In [ ]:
#上記のスクレイピングが終わるとurl_syosai_listsに１つ１つの物件のURLが入る。それを取り出し、要素を取得してく。
for j in tqdm(range(len(url_syosai_lists))):
    try:
        url=url_syosai_lists[j]
        urls=requests.get(url)
        time.sleep(1) #サーバーに負荷がかかるので１秒待つ。
        urls.encoding=urls.apparent_encoding
        soup=BeautifulSoup(urls.content,"html.parser")

        details_df_1=pd.read_html(url)[2]
        details_df_2=pd.read_html(url)[3]

        house_name.append(soup.find(class_="section_h1-header-title").text) #家の名前
        price.append(soup.find(class_="property_view_note-list").find_all("span")[0].text) #家賃
        kanrihi.append(soup.find(class_="property_view_note-list").find_all("span")[1].text.split("\xa0")[1]) #スクレイピングすると「\xa0」が混ざってしまうのでsplitして１個目を取得 #管理費
        sikikin.append(soup.find_all(class_="property_view_note-list")[1].find_all("span")[0].text.split("\xa0")[1]) #敷金
        reikin.append(soup.find_all(class_="property_view_note-list")[1].find_all("span")[1].text.split("\xa0")[1]) #礼金
        hosyokin.append(soup.find_all(class_="property_view_note-list")[1].find_all("span")[2].text.split("\xa0")[1]) #保証金
        sikibiki.append(soup.find_all(class_="property_view_note-list")[1].find_all("span")[3].text.split("\xa0")[1]) #敷引
        address.append(details_df_1.iloc[0,1:4][1]) #住所
        nearest_station.append(details_df_1.iloc[1,1:4][1]) #最寄り駅、徒歩
        layout.append(details_df_1.iloc[2,1:2][1]) #間取り。1LDKとか
        area.append(details_df_1.iloc[2,3:4][3]) #専有面積
        age.append(details_df_1.iloc[3,1:2][1]) #築年数
        floor.append(details_df_1.iloc[3,3:4][3]) #何階か
        direction.append(details_df_1.iloc[4,1:2][1]) #方角。南向きとか。
        house_type.append(details_df_1.iloc[4,3:4][3]) #マンションorアパートなど
        construction.append(details_df_2.iloc[0,3:4][3]) #鉄筋、木造など
        floor_max.append(details_df_2.iloc[1,1:2][1]) #何回まであるか。2階/5階など
        insurance.append(details_df_2.iloc[2,1:2][1]) #保証について
        parking.append(details_df_2.iloc[2,3:4][3]) #駐車場の有無
        # 設備については全て取り出すのは大変なので個人的に気になるやつを取得
        setsubi=soup.find_all(class_="inline_list")[1]
        text=str(setsubi)
        text=text.split("、")

        # バストイレ別なら「1」、別じゃないなら「0」。他も同様。
        if '<ul class="inline_list">\n<li>バストイレ別' in text:
            separate_bath.append(1)
        else:
            separate_bath.append(0)

        if "エレベーター" in text: #エレベーターの有無
            elevator.append(1)
        else:
            elevator.append(0)

        if "ペット" in text: #ペットOKか
            pet.append(1)
        else:
            pet.append(0)

        if "室内洗濯置" in text: #室内に洗濯機がおけるか
            washing_machin.append(1)
        else:
            washing_machin.append(0)

        if "オートロック" in text: #オートロックの有無
            auto_lock.append(1)
        else:
            auto_lock.append(0)

        if "洗面所独立" in text: #独立洗面台の有無
            washroom.append(1)
        else:
            washroom.append(0)

    except AttributeError:
        continue    

# 得られたリストを全てSeriesに変換、結合していく。
house_name_s=pd.Series(house_name)     
price_s=pd.Series(price)
kanrihi_s=pd.Series(kanrihi)
sikikin_s=pd.Series(sikikin)
reikin_s=pd.Series(reikin)
hosyokin_s=pd.Series(hosyokin)
sikibiki_s=pd.Series(sikibiki)
address_s=pd.Series(address)
nearest_station_s=pd.Series(nearest_station)
layout_s=pd.Series(layout)
area_s=pd.Series(area)
age_s=pd.Series(age)
floor_s=pd.Series(floor)
direction_s=pd.Series(direction)
house_type_s=pd.Series(house_type)
construction_s=pd.Series(construction)
floor_max_s=pd.Series(floor_max)
insurance_s=pd.Series(insurance)
parking_s=pd.Series(parking)
separate_bath_s=pd.Series(separate_bath)
elevator_s=pd.Series(elevator)
pet_s=pd.Series(pet)
washing_machin_s=pd.Series(washing_machin)
auto_lock_s=pd.Series(auto_lock)
washroom_s=pd.Series(washroom)

df=pd.concat([house_name_s,price_s,kanrihi_s,sikikin_s,reikin_s,hosyokin_s,sikibiki_s,\
              address_s,nearest_station_s,layout_s,area_s,age_s,floor_s,direction_s,house_type_s,\
              construction_s,floor_max_s,insurance_s,parking_s,separate_bath_s,\
              elevator_s,pet_s,washing_machin_s,auto_lock_s,washroom_s],axis=1)

df.columns=["house_name","price","kanrihi","sikikin","reikin","hosyokin","sikibiki","address",\
            "nearest_station","layout","area","age","floor","direction","house_type","construction","floor_max",\
           "insurance","parking","separate_bath","elevator","pet","washing_machin","auto_lock","washroom"]

df.to_csv('suumo.csv',header=True, index=False) # csvに保存